In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reload all modules imported with %aimport
%load_ext autoreload
%autoreload 1


In [2]:
import pandas as pd
idx = pd.IndexSlice

import datetime as dt
from datetime import date


from datetime import timedelta
import dateutil.relativedelta as rd
import dateutil.parser as dup

from trans.date_manip import Date_Manipulator

In [3]:
Memorialize = False

In [4]:
today = dt.datetime.combine( dt.date.today(), dt.time.min)
if Memorialize:
    today = dup.parse("03/09/2018")
    
start = dup.parse("01/01/2018")
end   = today



In [5]:
import trans.dataprovider.alphavantage as aa
%aimport trans.dataprovider.alphavantage

aa_dp = aa.Alphavantage()

In [6]:
import trans.dataprovider.odo as odo_reader
%aimport trans.dataprovider.odo

from sqlalchemy.ext.declarative import declarative_base

dburl="sqlite:////tmp/full.db"
decBase_r = declarative_base()

odr = odo_reader.ODO(dburl, declarative_base=decBase_r, provider=aa_dp)

df_aa = odr.get(tickers=["FB", "AAPL", "AMZN", "NFLX", "GOOG" ], start="2018-03-01")
df_aa.shape

(47, 40)

In [7]:
from trans.data import GetData
gd = GetData()

universe = gd.get_r1000_tickers()
universe.sort()
len(universe)

987

In [8]:
import trans.quantfactor.volatility as vf
%aimport trans.quantfactor.volatility

%aimport trans.gtrans
%aimport trans.quantfactor.base

v = vf.Volatility(universe=["FB", "AAPL", "AMZN", "NFLX", "GOOG" ], dataProvider=odr)

dataProvider passed
init: need to override


In [9]:
daily_price_df = v.load_prices(start=start, end=end)

dataProvider passed


In [10]:
daily_price_df.shape

(87, 48)

In [11]:
existing_tickers = list(daily_price_df.columns.get_level_values(1).unique())
existing_tickers.sort()
existing_tickers

['AAPL', 'AMZN', 'FB', 'GOOG', 'NFLX', 'SPY']

In [12]:
price_attr = "Adj Close"
ret_attr = "Pct"
daily_ret_df = v.create_dailyReturns(price_attr, ret_attr )

PANDA version > 0.20.0
PANDA version > 0.20.0


/home/ubuntu/Notebooks/trans/gtrans.py:902: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  concatted.sortlevel(axis=0, inplace=True)
/home/ubuntu/Notebooks/trans/gtrans.py:903: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  concatted.sortlevel(axis=1, inplace=True)


In [13]:
daily_ret_df.shape

(87, 12)

In [14]:
dm = Date_Manipulator( v.price_df.index )
eom_in_idx = dm.periodic_in_idx_end_of_month(end)
v.set_endDates( eom_in_idx )

In [15]:
from datetime import timedelta
vol_window = timedelta(days=30)
 
v.create_period_attr( v.daily_ret_df.loc[:, idx[ret_attr,:]], start, end, vol_window, "Volatility")

Volatility                                                  
                 AAPL      AMZN        FB      GOOG      NFLX       SPY
Dt                                                                     
2018-01-31   0.009462  0.009819  0.015685  0.007467  0.028084  0.005495
2018-02-28   0.021743  0.022927  0.023674  0.024222  0.028408  0.016685
2018-03-29   0.016277  0.020403  0.024933  0.022016  0.032206  0.012532
2018-04-30   0.016128  0.025275  0.026855  0.017441  0.029001  0.010767

In [16]:
daily_rank_df = v.create_ranks()

PANDA version > 0.20.0


In [17]:
daily_rank_df.loc[ "2018-01-31":"2018-03-02"]

Rank                    
           AAPL AMZN   FB GOOG NFLX
Dt                                 
2018-01-31  NaN  NaN  NaN  NaN  NaN
2018-02-01  2.0  3.0  4.0  1.0  5.0
2018-02-02  2.0  3.0  4.0  1.0  5.0
2018-02-05  2.0  3.0  4.0  1.0  5.0
2018-02-06  2.0  3.0  4.0  1.0  5.0
2018-02-07  2.0  3.0  4.0  1.0  5.0
2018-02-08  2.0  3.0  4.0  1.0  5.0
2018-02-09  2.0  3.0  4.0  1.0  5.0
2018-02-12  2.0  3.0  4.0  1.0  5.0
2018-02-13  2.0  3.0  4.0  1.0  5.0
2018-02-14  2.0  3.0  4.0  1.0  5.0
2018-02-15  2.0  3.0  4.0  1.0  5.0
2018-02-16  2.0  3.0  4.0  1.0  5.0
2018-02-20  2.0  3.0  4.0  1.0  5.0
2018-02-21  2.0  3.0  4.0  1.0  5.0
2018-02-22  2.0  3.0  4.0  1.0  5.0
2018-02-23  2.0  3.0  4.0  1.0  5.0
2018-02-26  2.0  3.0  4.0  1.0  5.0
2018-02-27  2.0  3.0  4.0  1.0  5.0
2018-02-28  2.0  3.0  4.0  1.0  5.0
2018-03-01  1.0  2.0  3.0  4.0  5.0
2018-03-02  1.0  2.0  3.0  4.0  5.0

In [18]:
factor_df = v.create_factor()

PANDA version > 0.20.0
PANDA version > 0.20.0


/home/ubuntu/Notebooks/trans/gtrans.py:902: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  concatted.sortlevel(axis=0, inplace=True)
/home/ubuntu/Notebooks/trans/gtrans.py:903: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  concatted.sortlevel(axis=1, inplace=True)


In [19]:
factor_df.tail()

Pct                                                         \
                AAPL AMZN   FB GOOG      NFLX      Port  Port < 0  Port > 0   
Dt                                                                            
2018-05-01 -0.023236  0.0  0.0  0.0  0.002688 -0.020548 -0.023236  0.002688   
2018-05-02 -0.044175 -0.0  0.0 -0.0  0.000192 -0.043984 -0.044175  0.000192   
2018-05-03 -0.001812  0.0 -0.0 -0.0 -0.005329 -0.007142 -0.001812 -0.005329   
2018-05-04 -0.039233  0.0  0.0  0.0  0.026950 -0.012284 -0.039233  0.026950   
2018-05-07 -0.007235  0.0  0.0  0.0  0.019276  0.012041 -0.007235  0.019276   

                         weight                    
            Port net SPY   AAPL AMZN FB GOOG NFLX  
Dt                                                 
2018-05-01 -0.020548 NaN     -1    0  0    0    1  
2018-05-02 -0.043984 NaN     -1    0  0    0    1  
2018-05-03 -0.007142 NaN     -1    0  0    0    1  
2018-05-04 -0.012284 NaN     -1    0  0    0    1  
2018-05-07  0.012041 NaN     -1    0  0    0    1

## Create residuals

### Create DataFrame for residual computation

In [20]:
resid_input_df =pd.concat( [ factor_df.loc[:, idx[ret_attr, "Port net"]],
    v.daily_ret_df.loc[:, idx[ret_attr,"SPY"]]
           ], axis=1
         )

In [21]:
%aimport trans.stacked.residual
from trans.stacked.residual import Residual

(resStart, resEnd) = (resid_input_df.index.min(), resid_input_df.index.max())
regWindow = rd.relativedelta(months=+2)
regStep   = rd.relativedelta(weeks=+4)

In [22]:
ret_attr

'Pct'

In [23]:
%aimport trans.reg

In [24]:
rstack = Residual(indCols=[ idx[ret_attr, "SPY"] ], debug=True)
rstack.init(df=resid_input_df, start=resStart, end=v.endDates[-1], window=regWindow, step=regStep)
resid_stack = rstack.repeated()
rstack.done()

DEPRECATED: <class 'trans.stacked.residual.Residual'>:__init__ called w/o "attr" arg., defaulting to Pct.
nextChunk for period 2018-03-01 00:00:00 to 2018-04-30 00:00:00 shape: (42, 2)
repeated: chunk label 2018-04-30 00:00:00 with shape (42, 2)


/home/ubuntu/Notebooks/trans/reg.py:217: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  df_big.sortlevel(axis=0, inplace=True)


IndCols: [('Pct', 'SPY')], depCol ('Pct', 'Port net'), cols [('Pct', 'SPY'), ('Pct', 'Port net')]
rps beta_df:                Beta 0    Beta 1
            Port net  Port net
Dt                            
2018-04-30  0.004172  0.841868
rps retAttr_df:             Contrib from 1 Contrib from SPY     Error Predicted
                 Port net         Port net  Port net  Port net
Dt                                                            
2018-04-24       0.004172        -0.011338 -0.015533 -0.007166
2018-04-25       0.004172         0.002081 -0.014714  0.006252
2018-04-26       0.004172         0.008558  0.010671  0.012730
2018-04-27       0.004172         0.000790 -0.000462  0.004962
2018-04-30       0.004172        -0.006474 -0.013564 -0.002303
nextChunk for period 2018-02-03 00:00:00 to 2018-04-02 00:00:00 shape: (39, 2)
repeated: chunk label 2018-04-02 00:00:00 with shape (39, 2)
IndCols: [('Pct', 'SPY')], depCol ('Pct', 'Port net'), cols [('Pct', 'SPY'), ('Pct', 'Port net')]
rps b

/home/ubuntu/Notebooks/trans/reg.py:218: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  df_big.sortlevel(axis=1, inplace=True)
/home/ubuntu/Notebooks/trans/gtrans.py:902: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  concatted.sortlevel(axis=0, inplace=True)
/home/ubuntu/Notebooks/trans/gtrans.py:903: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  concatted.sortlevel(axis=1, inplace=True)
/home/ubuntu/Notebooks/trans/gtrans.py:902: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  concatted.sortlevel(axis=0, inplace=True)
/home/ubuntu/Notebooks/trans/gtrans.py:903: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  concatted.sortlevel(axis=1, inplace=True)
/home/ubuntu/Notebooks/trans/reg.py:217: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  df_big.sortlevel(axis=0, inplace=True)
/home/ubuntu/Notebooks/trans/reg.py:218: FutureWarning: sortlevel is deprecated, u

In [25]:
(l, resid_last_df) = resid_stack[0]
resid_last_df.columns 

for r in resid_stack:
    (label, df) = r
    print("{l}: from {s} to {e}".format(l=label, s=df.index.min(), e=df.index.max()))

MultiIndex(levels=[['Contrib from 1', 'Contrib from SPY', 'Error', 'Predicted'], ['Port net']],
           labels=[[2], [0]])

2018-04-30 00:00:00: from 2018-03-01 00:00:00 to 2018-04-30 00:00:00
2018-04-02 00:00:00: from 2018-02-05 00:00:00 to 2018-04-02 00:00:00
2018-03-05 00:00:00: from 2018-01-08 00:00:00 to 2018-03-05 00:00:00


## Find the sensitivity of selected names

In [26]:
names = [ "BA", "PAGS", "CTXS", "TSLA", "NFLX"< "MSFT", "EA" ]

## Get returns

In [27]:
from trans.gtrans import *
# Get the data for the tickers in self.universe
price_df = GetDataProviderTransformer(names, cal_ticker="SPY", dataProvider=odr).fit_transform( pd.DataFrame())

# Limit the output to date range from start to end
price_df = price_df.loc[ start:, ]
        
price_df = price_df.loc[ :end, ]
price_attr = "Adj Close"
price_shifted_attr = price_attr + " prior"

daily_ret_pl = GenRetAttrTransformer( price_attr, price_shifted_attr, ret_attr, 1 )
daily_ret_df = daily_ret_pl.fit_transform( price_df )

dataProvider passed
PANDA version > 0.20.0
PANDA version > 0.20.0


/home/ubuntu/Notebooks/trans/gtrans.py:902: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  concatted.sortlevel(axis=0, inplace=True)
/home/ubuntu/Notebooks/trans/gtrans.py:903: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  concatted.sortlevel(axis=1, inplace=True)


## Compute betas wrt vol factor

In [28]:
volat_resid_df = resid_last_df.loc[:, idx["Error",:]]
volat_resid_df.tail()
lev0 =  volat_resid_df.columns.levels[0].tolist()
lev0[ lev0.index("Error")] = ret_attr

volat_resid_df.columns.set_levels(["Volat factor"], level=1, inplace=True)
volat_resid_df.columns.set_levels(lev0, level=0, inplace=True)

volat_resid_df.tail()

,Error
,Port net
Dt,
2018-04-24,-0.015533
2018-04-25,-0.014714
2018-04-26,0.010671
2018-04-27,-0.000462
2018-04-30,-0.013564


,Pct
,Volat factor
Dt,
2018-04-24,-0.015533
2018-04-25,-0.014714
2018-04-26,0.010671
2018-04-27,-0.000462
2018-04-30,-0.013564


In [29]:
reg_input_df = DataFrameConcat( [ daily_ret_df.loc[:, idx[ret_attr,:]], volat_resid_df] ).fit_transform(pd.DataFrame())

/home/ubuntu/Notebooks/trans/gtrans.py:902: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  concatted.sortlevel(axis=0, inplace=True)
/home/ubuntu/Notebooks/trans/gtrans.py:903: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  concatted.sortlevel(axis=1, inplace=True)


In [30]:
reg_input_df.columns
reg_input_df.loc[ volat_resid_df.index.max() - rd.relativedelta(months=+1):]

MultiIndex(levels=[['Pct'], ['BA', 'CTXS', 'EA', 'SPY', 'TSLA', 'Volat factor']],
           labels=[[0, 0, 0, 0, 0, 0], [0, 1, 2, 3, 4, 5]],
           names=[None, 'Ticker'])

Pct                                                     
Ticker            BA      CTXS        EA       SPY      TSLA Volat factor
Dt                                                                       
2018-04-02 -0.016591 -0.015841 -0.020785 -0.021585 -0.051291    -0.030434
2018-04-03  0.025989  0.005913  0.006654  0.012817  0.059609    -0.013162
2018-04-04 -0.010217  0.008926  0.005021  0.010699  0.072553    -0.013723
2018-04-05  0.027364  0.000216  0.009325  0.007892  0.065449    -0.000341
2018-04-06 -0.030559 -0.010031 -0.023674 -0.022286 -0.021000     0.022752
2018-04-09 -0.011162  0.005230  0.004224  0.004928 -0.032208    -0.014500
2018-04-10  0.038297  0.012356  0.015565  0.015900  0.051923    -0.008300
2018-04-11 -0.022310  0.002463 -0.015989 -0.005242 -0.012373     0.023705
2018-04-12  0.030914  0.009826  0.019448  0.008227 -0.022763    -0.002581
2018-04-13 -0.024298  0.003702 -0.004625 -0.002933  0.021287     0.002670
2018-04-16  0.007562  0.011275  0.013773  0.008222 -0.030399    -0.029749
2018-04-17  0.014920  0.016151  0.017350  0.010698 -0.012087     0.064942
2018-04-18  0.012711  0.007178  0.004988  0.000740  0.019674    -0.007133
2018-04-19 -0.001056 -0.010181 -0.014568 -0.005548  0.022942     0.023398
2018-04-20 -0.005783 -0.006686 -0.018033 -0.008479 -0.032791     0.029121
2018-04-23  0.000502  0.004763 -0.010671 -0.000150 -0.023670    -0.028851
2018-04-24 -0.028863 -0.004432 -0.008445 -0.013467  0.000318    -0.015533
2018-04-25  0.041938 -0.000207 -0.016106  0.002472 -0.009772    -0.014714
2018-04-26 -0.000204  0.047525  0.025026  0.010166  0.017065     0.010671
2018-04-27 -0.005572  0.020757 -0.017809  0.000939  0.030107    -0.000462
2018-04-30 -0.021474 -0.003486  0.004342 -0.007691 -0.000595    -0.013564
2018-05-01 -0.012052  0.017491  0.015681  0.001777  0.020483          NaN
2018-05-02 -0.016235 -0.002865 -0.007427 -0.006717  0.004101          NaN
2018-05-03  0.020050  0.003831  0.010341 -0.002204 -0.055454          NaN
2018-05-04  0.011310  0.004007  0.028876  0.012946  0.033890          NaN
2018-05-07  0.017941  0.012829  0.002265  0.003383  0.029515          NaN

In [31]:
from trans.regpipe import RegPipe
rp = RegPipe( reg_input_df )
rp.indCols( [ idx[ret_attr, "Volat factor"] ] )
rp.regress( resStart, v.endDates[-1], regWindow, regStep)

DEPRECATED: <class 'trans.regpipe.RegPipe'>:__init__ called w/o "attr" arg., defaulting to Pct.
IndCols: [('Pct', 'Volat factor')], depCol ('Pct', 'BA'), cols [('Pct', 'Volat factor'), ('Pct', 'BA')]
Fit: the following columns have naN:  ['Volat factor']
Fit: the following columns have naN:  ['Volat factor']
IndCols: [('Pct', 'Volat factor')], depCol ('Pct', 'CTXS'), cols [('Pct', 'Volat factor'), ('Pct', 'CTXS')]
Fit: the following columns have naN:  ['Volat factor']
Fit: the following columns have naN:  ['Volat factor']
IndCols: [('Pct', 'Volat factor')], depCol ('Pct', 'EA'), cols [('Pct', 'Volat factor'), ('Pct', 'EA')]
Fit: the following columns have naN:  ['Volat factor']
Fit: the following columns have naN:  ['Volat factor']
IndCols: [('Pct', 'Volat factor')], depCol ('Pct', 'SPY'), cols [('Pct', 'Volat factor'), ('Pct', 'SPY')]
Fit: the following columns have naN:  ['Volat factor']
Fit: the following columns have naN:  ['Volat factor']
IndCols: [('Pct', 'Volat factor')], depCol

/home/ubuntu/Notebooks/trans/reg.py:217: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  df_big.sortlevel(axis=0, inplace=True)
/home/ubuntu/Notebooks/trans/reg.py:218: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  df_big.sortlevel(axis=1, inplace=True)


In [32]:
rp.beta_df

Beta 0                                           Beta 1  \
                  BA      CTXS        EA       SPY      TSLA       BA   
Dt                                                                      
2018-03-05       NaN       NaN       NaN       NaN       NaN      NaN   
2018-04-02       NaN       NaN       NaN       NaN       NaN      NaN   
2018-04-30 -0.001736  0.002742 -0.000977 -0.000473 -0.003094  0.01709   

                                                       
                CTXS        EA           SPY     TSLA  
Dt                                                     
2018-03-05       NaN       NaN           NaN      NaN  
2018-04-02       NaN       NaN           NaN      NaN  
2018-04-30  0.051094  0.043842 -5.085472e-18  0.02248

In [33]:
missing_universe = list(set(universe) - set(existing_tickers))
missing_universe.sort()
print("No data for following R1000 names: {}".format(", ".join(missing_universe)))

universe = list(set(universe) - set(missing_universe))
universe.sort()
print("Available universe has {} tickers".format(len(universe)))

No data for following R1000 names: A, AA, AAL, AAP, ABBV, ABC, ABMD, ABT, ACAD, ACC, ACGL, ACHC, ACM, ACN, ADBE, ADI, ADM, ADP, ADS, ADSK, AEE, AEP, AES, AET, AFG, AFL, AFSI, AGCO, AGIO, AGN, AGNC, AGO, AGR, AHL, AIG, AIV, AIZ, AJG, AKAM, AKRX, AL, ALB, ALGN, ALK, ALKS, ALL, ALLE, ALLY, ALNY, ALR, ALSN, ALXN, AMAT, AMCX, AME, AMG, AMGN, AMH, AMP, AMSG, AMT, AMTD, AN, ANAT, ANET, ANSS, ANTM, AON, AOS, APA, APAM, APC, APD, APH, APLE, AR, ARE, ARMK, ARRS, ARW, ASB, ASH, ATHN, ATO, ATR, ATVI, AVB, AVGO, AVT, AVY, AWH, AWI, AWK, AXP, AXS, AXTA, AYI, AZO, BA, BAC, BAH, BAX, BBBY, BBT, BBY, BC, BCR, BDN, BDX, BEAV, BEN, BERY, BF.A, BF.B, BG, BHI, BIIB, BIO, BK, BKD, BKFS, BKU, BLK, BLL, BMRN, BMS, BMY, BOH, BOKF, BPOP, BR, BRCD, BRKR, BRO, BRX, BSX, BUFF, BURL, BWA, BWXT, BXP, C, CA, CAA, CAB, CABO, CACC, CAG, CAH, CAR, CASY, CAT, CB, CBG, CBI, CBOE, CBS, CBSH, CBT, CCI, CCK, CCL, CCO, CCP, CDK, CDNS, CDW, CE, CELG, CERN, CF, CFG, CFR, CFX, CGNX, CHD, CHH, CHK, CHRW, CHTR, CI, CIM, CINF, CIT,